In [1]:
%pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 85.2 MB/s eta 0:00:00


## 1. Data Loading in PennyLane

In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 1. Load the dataset
data = pd.read_csv('/content/QMLChallenge/data/challenge_higgs_data.csv')

X = data.iloc[:, 1:].values  # 28 features per sample
y = data.iloc[:, 0].values   # 1 (signal), 0 (background)

# 2. Feature Scaling (z-score, then map to [0, π] for angle encoding)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_angle = (X_scaled - X_scaled.min()) / (X_scaled.max() - X_scaled.min()) * np.pi


## 2. Train/Validation Split

In [21]:
X_train, X_val, y_train, y_val = train_test_split(
    X_angle, y, test_size=0.2, random_state=42, stratify=y
)


Quantum circuits typically require input data normalized to a specific range (e.g., [0, π] or [−π, π]), so you might map or scale your features accordingly before encoding

##Circuit Restrictions

* Max qubits per circuit: 20

* Main focus: The architecture must feature a quantum neural network (QNN). Hybrid models (classical + quantum) are allowed only if the quantum part is central.

* Framework: Use PennyLane; any library available in Google Colab is allowed.

* Don’t modify the final cell that plots the ROC curve.

* Input dataset: Use only the provided subset for training; the test set is reserved for final evaluation.

## 3. Quantum Neural Network (QNN) Design

### A. Building Blocks
We’ll need:

* Quantum circuit device: On simulator or hardware plugin (here, use the default qubit simulator).

* Feature encoding: Embed (part of) your data into the circuit (e.g., via angle encoding or amplitude encoding).

* Variational layers: Parameterized gates whose weights are learned during training.

* Measurement: Output for each sample is usually the expectation value of a Pauli operator.

### B. Pennylane Utility

In [22]:
import pennylane as qml
from pennylane import numpy as np

n_qubits = 6  # Or 8 for a larger model, but <=20 as per challenge rules
dev = qml.device("default.qubit", wires=n_qubits)

def angle_encoding(features):
    # Truncate or select first n_qubits features
    for i in range(n_qubits):
        qml.RY(features[i], wires=i)

def variational_block(weights):
    # One trainable rotation and controlled entangling layer per qubit
    for i in range(n_qubits):
        qml.RY(weights[i], wires=i)
    # Entangle neighboring qubits
    for i in range(n_qubits-1):
        qml.CNOT(wires=[i, i+1])
    qml.CNOT(wires=[n_qubits-1, 0])

@qml.qnode(dev)
def qnn_circuit(features, weights):
    angle_encoding(features)
    variational_block(weights[0])
    variational_block(weights[1])
    return qml.expval(qml.PauliZ(0))

## 4. Classifier Wrapper and Training

In [23]:
def classifier(params, x):
    weights = params[:-1]
    bias = params[-1]
    return qnn_circuit(x, weights) + bias

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def binary_cross_entropy(preds, targets):
    preds = np.clip(sigmoid(preds), 1e-8, 1-1e-8)
    return -np.mean(targets * np.log(preds) + (1 - targets) * np.log(1 - preds))

def accuracy(preds, targets):
    pred_labels = (sigmoid(preds) > 0.5).astype(int)
    return np.mean(pred_labels == targets)

### Training Loop

In [24]:
from pennylane.optimize import AdamOptimizer

# Prepare parameters
init_weights = np.random.randn(n_qubits)
init_bias = 0.0
params = np.append(init_weights, init_bias)

opt = AdamOptimizer(stepsize=0.01)
num_epochs = 20
batch_size = 16

for epoch in range(num_epochs):
    batch_index = np.random.randint(0, len(X_train), batch_size)
    X_batch = X_train[batch_index]
    y_batch = y_train[batch_index]

    def cost(params):
        preds = np.array([classifier(params, x) for x in X_batch])
        return binary_cross_entropy(preds, y_batch)

    params = opt.step(cost, params)

    # Evaluate
    train_loss = cost(params)
    val_preds = np.array([classifier(params, x) for x in X_val[:100]])
    val_loss = binary_cross_entropy(val_preds, y_val[:100])
    val_acc = accuracy(val_preds, y_val[:100])
    print(f"Epoch {epoch+1:2d}  Loss: {train_loss:.4f}  Val Loss: {val_loss:.4f}  Val Acc: {val_acc:.3f}")

# Save weights as required
np.save('trained_weights.npy', params[:-1])


Epoch  1  Loss: 0.7116  Val Loss: 0.7017  Val Acc: 0.460
Epoch  2  Loss: 0.7262  Val Loss: 0.7005  Val Acc: 0.460
Epoch  3  Loss: 0.6987  Val Loss: 0.6996  Val Acc: 0.480
Epoch  4  Loss: 0.7001  Val Loss: 0.6988  Val Acc: 0.470
Epoch  5  Loss: 0.6877  Val Loss: 0.6980  Val Acc: 0.460
Epoch  6  Loss: 0.6961  Val Loss: 0.6973  Val Acc: 0.470
Epoch  7  Loss: 0.6899  Val Loss: 0.6968  Val Acc: 0.500
Epoch  8  Loss: 0.6830  Val Loss: 0.6964  Val Acc: 0.480
Epoch  9  Loss: 0.6866  Val Loss: 0.6961  Val Acc: 0.460
Epoch 10  Loss: 0.6911  Val Loss: 0.6959  Val Acc: 0.460
Epoch 11  Loss: 0.6880  Val Loss: 0.6958  Val Acc: 0.440
Epoch 12  Loss: 0.6880  Val Loss: 0.6957  Val Acc: 0.440
Epoch 13  Loss: 0.6943  Val Loss: 0.6954  Val Acc: 0.450
Epoch 14  Loss: 0.6871  Val Loss: 0.6954  Val Acc: 0.450
Epoch 15  Loss: 0.6879  Val Loss: 0.6955  Val Acc: 0.460
Epoch 16  Loss: 0.6921  Val Loss: 0.6956  Val Acc: 0.460
Epoch 17  Loss: 0.6861  Val Loss: 0.6956  Val Acc: 0.450
Epoch 18  Loss: 0.6951  Val Los

In [25]:
def variational_classifier(x, weights, bias):
    return qnn_circuit(x, weights) + bias

# Loss: Mean squared error or cross-entropy (if appropriate)
def square_loss(predictions, labels):
    return np.mean((predictions - labels)**2)